In this notebook, I will be discussing how I created the project, explaining certain code, and the thought process on it

## Planning

My plan for the project is to utilize cheapshark API to extract different stores and game prices that I have in my library. This project is my first time utilizing API Systems and understading how to extract data from a JSON.

## Execution

In order to understand the API, I read through the documentation and found you can extract a games data if you can provide either the name or the Game ID CheapShark. I decided to utilize CheapShark ID to find my games but I didn't know the game IDs so I needed to look for there game IDS. Now, as a programmer, we should make things simplier but the mathamatics side said "we're gonna make this tough".  I found that you can search up their games ids by using their API so I created a function where I use a part of the games name and extract the title, id's ,and steamID's of the game.  

In [13]:
#Import the necessary packages for the project
import requests
import pandas as pd
import datetime
import time
pd.options.mode.chained_assignment = None

In [2]:
def game_search(gametitle):
    game=[]
    games='https://www.cheapshark.com/api/1.0/games?title='+gametitle
    
    #Request the url
    r= requests.get(games)

    #Reads the json file
    json_game=r.json()
    
    # Iterates the len of json_data and extract the store id and store name to create a datatable 
    for i in range(len(json_game)):
        dict={"gameID":json_game[i]['gameID'],
              "steamAppID":json_game[i]['steamAppID'],
             "external_name":json_game[i]['external']}
        game.append(dict)
    game_id= pd.DataFrame(game)
    return game_id

In [4]:
game_search('Monster Hunter').head(5)

,gameID,steamAppID,external_name
0,235239,1446780,MONSTER HUNTER RISE
1,189379,582010,MONSTER HUNTER: WORLD
2,287215,2246340,Monster Hunter Wilds
3,285429,2356560,Monster Hunter Stories
4,242048,None,MONSTER HUNTER RISE: Sunbreak


By the end of this, I basically made "search engine" of any games I can search. This function helped me find the different games in my library and their IDs. I collected this information into a notebook and later a list for future functions

###  What I learned

- Using indexing to extract the needed data
- Using "requests" an ".json()" to have the JSON File
- Creating a function to make the process repetable to find the other games' ids 

## Planning 

With the list_ids at hand, now we need to use the API to extract game_name, ids, deals, retail price and original price. In addition, I wanted the dataset to have, the date when it was extracted and the store name

## Execution

This section is the most complicated part of the project. Here, I read the documentation and tinker the code to extract the data. This is the end product after 2-3 hours of tinkering and testing code on how to extract data from one game and using for loops for multiple games. An issue I found was how the data I was extracting didn't have the store names instead it had the store IDs. This was an unfortunate detail but not world ending because I can create a seperate dataframe with stores and their store id's and join the names with the main data frame. After that, I can extract the data frame to a csv so I can store the file into my device.

In [6]:
list_ids=['157711','220845','252306','270964','216932','170812','160759','152601','287841','235239','189379']

In [7]:
json_id=[]
for i in range(len(list_ids)):
    id='https://www.cheapshark.com/api/1.0/games?id='+list_ids[i]
    r=requests.get(id)
    json=r.json()
    json_id.append(json)

In [8]:
price=[]
for i in range(11):
    for j in range(len(json_id[i]['deals'])):
        dict={"Time": datetime.date.today(),
            "title":json_id[i]['info']['title'],
            "steamAppID":json_id[i]['info']['steamAppID'],
             "storeID":json_id[i]['deals'][j]['storeID'],
             "price":json_id[i]['deals'][j]['price'] ,
             "retailprice":json_id[i]['deals'][j]['retailPrice'],
             'savings':json_id[i]['deals'][j]['savings']}
        price.append(dict)
               
# Creating the price dataframe
price_01=pd.DataFrame(price)
price_01

,Time,title,steamAppID,storeID,price,retailprice,savings
0,2025-03-17,NieR: Automata,524220,1,15.99,39.99,60.015004
1,2025-03-17,NieR Replicant ver.1.22474487139...,1113560,1,23.99,59.99,60.010002
2,2025-03-17,NieR Replicant ver.1.22474487139...,1113560,11,23.99,59.99,60.010002
3,2025-03-17,NieR Replicant ver.1.22474487139...,1113560,3,24.00,59.99,59.993332
4,2025-03-17,NieR Replicant ver.1.22474487139...,1113560,30,28.98,59.99,51.691949
...,...,...,...,...,...,...,...
90,2025-03-17,MONSTER HUNTER: WORLD,582010,29,29.99,29.99,0.000000
91,2025-03-17,MONSTER HUNTER: WORLD,582010,34,29.99,29.99,0.000000
92,2025-03-17,MONSTER HUNTER: WORLD,582010,2,29.99,29.99,0.000000
93,2025-03-17,MONSTER HUNTER: WORLD,582010,24,29.99,29.99,0.000000


In [9]:
def store():
    store='https://www.cheapshark.com/api/1.0/stores'
    #Request the api for information 
    r= requests.get(store)

    #Reads the json file
    json_store=r.json()
    store=[]

    # Iterates the len of json_data and extract the store id and store name to create a datatable 
    for i in range(len(json_store)):
        dict={"storeID":json_store[i]['storeID'],"storeName":json_store[i]['storeName']}
        store.append(dict)

    #Creating a DataFrame using a list of Dictionaries 
    store_id=pd.DataFrame(store)
    return store_id

In [10]:
store_id=store()
store_id.head()

,storeID,storeName
0,1,Steam
1,2,GamersGate
2,3,GreenManGaming
3,4,Amazon
4,5,GameStop


In [11]:
sales=  pd.merge(price_01,store_id)
sales.head()

,Time,title,steamAppID,storeID,price,retailprice,savings,storeName
0,2025-03-17,NieR: Automata,524220,1,15.99,39.99,60.015004,Steam
1,2025-03-17,NieR Replicant ver.1.22474487139...,1113560,1,23.99,59.99,60.010002,Steam
2,2025-03-17,Persona 5 Royal,1687950,1,23.99,59.99,60.010002,Steam
3,2025-03-17,Persona 3 Reload,2161700,1,34.99,69.99,50.007144,Steam
4,2025-03-17,Persona 4 Golden,1113000,1,11.99,19.99,40.020010,Steam


In [14]:
timestr = time.strftime("%Y_%m_%d")
print (timestr)
sales.to_csv('sales_'+timestr+'.csv', index=False) 

2025_03_17


### What I learned

- Merging data frames
- Using "requests" an ".json()" to have the JSON File
- Utilizing grouped loops to extract data from different sections of the JSON file
- Integrated time into the data and utilize the date in the names of the file
- Exported CSV files from a Dataframe